# Jupyter Example
## LAMMPS: granregion/in.granregion.funnel
Reference: https://github.com/lammps/lammps/blob/master/examples/granregion/in.granregion.funnel  
Required files: None  
Description: pour particles into cone-shaped funnel, settle them, let them run out bottom

In [1]:
from __future__ import print_function
import random, math
from lammps import IPyLammps

In [2]:
L = IPyLammps()

L.variable("name string funnel_pour")
L.thermo_modify("flush yes")
L.units("si")
L.variable("PI equal 3.141592653589")
L.variable("seed equal 14314")

# Geometry-related parameters
L.variable("xlo equal 10")
L.variable("xhi equal 40")
L.variable("ylo equal 10")
L.variable("yhi equal 40")
L.variable("zlo equal -20")
L.variable("zhi equal 50")
L.variable("xc equal 25")
L.variable("yc equal 25")
L.variable("zconehi equal 50")
L.variable("zconelo equal 10")
L.variable("zcyllo equal 0")
L.variable("radconelo equal 2")
L.variable("radconehi equal 20")

# Particle sizes
L.variable("rlo equal 0.25")
L.variable("rhi equal 0.5")
L.variable("dlo equal 2.0*${rlo}")
L.variable("dhi equal 2.0*${rhi}")
L.variable("skin equal ${rhi}")

# Granular contact parameters
L.variable("coeffRes equal 0.1")
L.variable("coeffFric equal 0.5")
L.variable("density equal 1.0")
L.variable("EYoung equal 10^5")
L.variable("Poisson equal 2.0/7.0")
L.variable("GShear equal ${EYoung}/(2*(1+${Poisson}))")
L.variable("gravity equal 1.0")
L.variable("reff equal 0.5*(${rhi}+${rlo})")
L.variable("meff equal ${density}*4.0/3.0*${PI}*${reff}^3")
L.variable("min_mass equal ${density}*4.0/3.0*${PI}*${rlo}*${rlo}*${rlo}")
L.variable("max_mass equal ${density}*4.0/3.0*${PI}*${rhi}*${rhi}*${rhi}")

# Typical way to set kn, kt, etc.
L.variable("kn equal 4.0*${GShear}/(3*(1-${Poisson}))")
L.variable("kt equal 4.0*${GShear}/(2-${Poisson})")
L.variable("a equal (-2.0*log(${coeffRes})/${PI})^2")
L.variable("gamma_n equal sqrt($a*2*${kn}/${min_mass}/(1+0.25*$a))")
L.variable("gamma_t equal ${gamma_n}*0.5")
L.variable("tcol equal ${PI}/sqrt(2*${kn}/${min_mass}-${gamma_n}/4.0)")
L.variable("dt equal ${tcol}*0.05")
L.timestep("${dt}")

LAMMPS output is captured by PyLammps wrapper


In [3]:
L.variable("dumpfreq equal 10000")
L.variable("logfreq equal 1000")
L.newton("off")
L.atom_style("sphere")
L.boundary("p p f")
L.region("boxreg block ${xlo} ${xhi} ${ylo} ${yhi} ${zlo} ${zhi}")
L.create_box("1 boxreg")
L.pair_style("gran/hertz/history ${kn} ${kt} ${gamma_n} ${gamma_t} ${coeffFric} 1")
L.pair_coeff("* *")
L.neighbor("${skin} bin")
L.thermo("${logfreq}")
L.comm_style("brick")
L.comm_modify("mode multi group all vel yes")
L.balance("1.1 shift xyz 20 1.1")
L.fix("bal all balance 10000 1.1 shift xyz 20 1.01")

In [4]:
# Options specific to pouring

# insertion region for fix/pour
L.region("insreg cylinder z ${xc} ${yc} 10 30 50 side in units box")

# Define cone and cylinder regions - see lammps doc on region command

# Top is open
L.region("cylreg cylinder z ${xc} ${yc} ${radconelo} ${zcyllo} ${zconelo} side in units box open 2")

# Bottom and top are open
L.region("conereg cone z ${xc} ${yc} ${radconelo} ${radconehi} ${zconelo} ${zconehi} side in units box open 1 open 2")
          
L.region("hopreg union 2 conereg cylreg")
L.fix("grav all gravity ${gravity} vector 0 0 -1")
L.fix("1 all nve/sphere")
L.fix("hopper3 all wall/gran/region hertz/history ${kn} ${kt} ${gamma_n} ${gamma_t} ${coeffFric} 1 region hopreg")
L.fix("ins all pour 2000 1 42424 region insreg diam range ${dlo} ${dhi} dens ${density} ${density}")

'Particle insertion: 3000 every 59965 steps, 2000 by step 1'

In [5]:
# Output
L.dump("1 all custom ${dumpfreq} ${name}.dump id type mass diameter x y z")

#L.dump("2 all image 4000 image.*.jpg type type axes yes 0.8 0.02 view 60 -30 zoom 3.0 box no 0.0 axes no 0.0 0.0")
#L.dump_modify("2 pad 6")

L.dump("3 all movie 5000 mov.mp4 type type axes yes 0.8 0.02 view 60 -30 zoom 3.0 box no 0.0 axes no 0.0 0.0")
L.dump_modify("3 pad 6")

L.thermo_style("custom step cpu atoms ke")
L.thermo_modify("flush yes lost warn")

In [6]:
# Initial run to fill up the cone
L.run("20000")
L.unfix("ins")
L.run("150000")

['Setting up Verlet run ...',
 '  Unit style    : si',
 '  Current step  : 20000',
 '  Time step     : 0.000105472',
 'Per MPI rank memory allocation (min/avg/max) = 14.33 | 14.33 | 14.33 Mbytes',
 'Step CPU Atoms KinEng ',
 '   20000            0     2000    6443.7665 ',
 '   21000   0.19603801     2000    6572.3531 ',
 '   22000   0.38649797     2000    6723.8376 ',
 '   23000   0.57785296     2000    6853.1812 ',
 '   24000   0.76907396     2000    6976.0209 ',
 '   25000   0.96972394     2000    7096.9955 ',
 '   26000    1.1641328     2000    7215.5795 ',
 '   27000    1.3580129     2000    7349.2382 ',
 '   28000     1.552469     2000    7471.8719 ',
 '   29000    1.7483399     2000    7574.8228 ',
 '   30000     1.960753     2000    7659.3836 ',
 '   31000    2.1591699     2000    7703.6856 ',
 '   32000    2.3615489     2000     7644.279 ',
 '   33000     2.567683     2000    7526.6944 ',
 '   34000     2.776783     2000     7370.082 ',
 '   35000     3.021044     2000    7193.

In [7]:
# remove "plug" - need to redefine cylinder region & union
L.region("cylreg delete")
L.region("hopreg delete")

# Bottom and top are open
L.region("cylreg cylinder z ${xc} ${yc} ${radconelo} ${zcyllo} ${zconelo} side in units box open 1 open 2")

L.region("hopreg union 2 cylreg conereg")
L.unfix("hopper3")
L.fix("hopper3 all wall/gran/region hertz/history ${kn} ${kt} ${gamma_n} ${gamma_t} ${coeffFric} 1 region hopreg")

In [8]:
L.run("600000")

['Setting up Verlet run ...',
 '  Unit style    : si',
 '  Current step  : 170000',
 '  Time step     : 0.000105472',
 'Per MPI rank memory allocation (min/avg/max) = 14.33 | 14.33 | 14.33 Mbytes',
 'Step CPU Atoms KinEng ',
 '  170000            0     2000    3.6839579 ',
 '  171000   0.44973493     2000    4.4723212 ',
 '  172000   0.89643312     2000     5.412119 ',
 '  173000     1.338063     2000    6.0613468 ',
 '  174000    1.7774251     2000    7.0007387 ',
 '  175000    2.2250471     2000     9.166075 ',
 '  176000    2.6767199     2000    11.767734 ',
 '  177000    3.1527381     2000    14.302491 ',
 '  178000    3.6223531     2000    17.901646 ',
 '  179000    4.0775599     2000     22.45433 ',
 '  180000     4.542006     2000    27.813179 ',
 '  181000    4.9900839     2000    33.985204 ',
 '  182000    5.4291689     2000    40.791424 ',
 '  183000    5.8640089     2000    48.071974 ',
 '  184000    6.2961531     2000    55.944526 ',
 '  185000     6.734859     2000    64.4

In [9]:
L.undump(3)
L.video("mov.mp4")